In [9]:
import json
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

# BERT 모델 로드
model = SentenceTransformer('all-MiniLM-L6-v2')

# JSON 파일에서 앱 데이터 불러오기
def load_app_data(json_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        app_data = json.load(f)
    return pd.DataFrame(app_data)

# 클린된 앱 데이터 로드
def load_cleaned_apps_data(csv_file):
    return pd.read_csv(csv_file)

# app_data와 cleaned_data 결합하여 description 결합
def merge_app_descriptions(app_data, cleaned_data):
    merged_data = pd.merge(app_data, cleaned_data, left_on="packageName", right_on="appId")
    merged_data = merged_data[['packageName', 'description']]
    return merged_data

# 클러스터링 수행 및 결과 반환
def perform_clustering(merged_data, n_clusters=4):
    descriptions = merged_data['description'].fillna('').tolist()

    # Sentence-BERT 모델을 사용해 앱 설명을 임베딩으로 변환
    embeddings = model.encode(descriptions)
    
    # 실시간 KMeans 클러스터링
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    clusters = kmeans.fit_predict(embeddings)

    # 클러스터 결과를 데이터에 추가
    merged_data['cluster'] = clusters

    # 클러스터별 중심점과 각 설명 임베딩 간의 유사도 계산하여 클러스터 이름 생성
    cluster_names = []
    for cluster_num in range(n_clusters):
        # 해당 클러스터의 설명 임베딩
        cluster_embeddings = embeddings[merged_data['cluster'] == cluster_num]
        # 해당 클러스터의 중심
        cluster_center = kmeans.cluster_centers_[cluster_num].reshape(1, -1)
        
        # 중심과 가장 유사한 설명 선택
        similarities = cosine_similarity(cluster_embeddings, cluster_center)
        best_index = similarities.argmax()

        # 'merged_data'의 인덱스를 사용하여 설명 추출
        best_description = merged_data.loc[merged_data['cluster'] == cluster_num].iloc[best_index]['description']
        cluster_names.append(best_description[:50])  # 클러스터 이름을 설명 일부로 사용

    # 최종 데이터 프레임에 클러스터 이름 매핑
    merged_data['cluster_name'] = merged_data['cluster'].map(lambda x: cluster_names[x])

    # 필요한 결과 형식으로 반환
    result_data = merged_data[['packageName', 'cluster_name']]
    return result_data

# 실행 예제
if __name__ == "__main__":
    # JSON 파일과 CSV 파일 경로
    json_file = 'app_data_ee21702d85b4e078.json'
    csv_file = 'cleaned_apps_data.csv'
    
    # 앱 데이터 및 클린 데이터 로드
    app_data = load_app_data(json_file)
    cleaned_data = load_cleaned_apps_data(csv_file)
    
    # 앱 데이터와 클린 데이터 병합 및 클러스터링 수행
    merged_data = merge_app_descriptions(app_data, cleaned_data)
    result = perform_clustering(merged_data, n_clusters=4)
    
    # 결과 출력
    for item in result.itertuples():
        print(f"Package Name: {item.packageName}, Cluster Name: {item.cluster_name}")


c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_68124\62434106.py:18: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(csv_file)


Package Name: com.selabs.speak, Cluster Name: With the official ChatGPT app, get instant answers
Package Name: com.selabs.speak, Cluster Name: With the official ChatGPT app, get instant answers
Package Name: com.selabs.speak, Cluster Name: With the official ChatGPT app, get instant answers
Package Name: com.selabs.speak, Cluster Name: With the official ChatGPT app, get instant answers
Package Name: com.selabs.speak, Cluster Name: With the official ChatGPT app, get instant answers
Package Name: com.selabs.speak, Cluster Name: With the official ChatGPT app, get instant answers
Package Name: com.selabs.speak, Cluster Name: With the official ChatGPT app, get instant answers
Package Name: com.selabs.speak, Cluster Name: With the official ChatGPT app, get instant answers
Package Name: com.selabs.speak, Cluster Name: With the official ChatGPT app, get instant answers
Package Name: com.selabs.speak, Cluster Name: With the official ChatGPT app, get instant answers
Package Name: com.selabs.speak

c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [11]:
import json
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction import text

# BERT 모델 로드
model = SentenceTransformer('all-MiniLM-L6-v2')

# JSON 파일에서 앱 데이터 불러오기
def load_app_data(json_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        app_data = json.load(f)
    return pd.DataFrame(app_data)

# 클린된 앱 데이터 로드
def load_cleaned_apps_data(csv_file):
    return pd.read_csv(csv_file)

# app_data와 cleaned_data 결합하여 description 결합
def merge_app_descriptions(app_data, cleaned_data):
    merged_data = pd.merge(app_data, cleaned_data, left_on="packageName", right_on="appId")
    merged_data = merged_data[['packageName', 'description']]
    return merged_data

# 앱 이름을 불용어 처리
def remove_stopwords(app_data, additional_stopwords):
    # 앱 이름을 불용어 리스트에 추가
    app_names = app_data['packageName'].apply(lambda x: x.lower()).tolist()
    stop_words = list(text.ENGLISH_STOP_WORDS.union(app_names).union(additional_stopwords))
    return stop_words

# 클러스터링 수행 및 결과 반환
def perform_clustering(merged_data, n_clusters=4, additional_stopwords=[]):
    descriptions = merged_data['description'].fillna('').tolist()

    # 앱 이름을 불용어 처리하여 TF-IDF에 사용
    stop_words = remove_stopwords(merged_data, additional_stopwords)

    # TF-IDF 벡터화
    vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=1000)  # 불용어 처리 및 피처 수 제한
    tfidf_matrix = vectorizer.fit_transform(descriptions)
    
    # KMeans 클러스터링
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    clusters = kmeans.fit_predict(tfidf_matrix)

    # 클러스터 결과를 데이터에 추가
    merged_data['cluster'] = clusters

    # 클러스터별 중심점과 각 설명 임베딩 간의 유사도 계산하여 클러스터 이름 생성
    cluster_names = []
    for cluster_num in range(n_clusters):
        cluster_descriptions = tfidf_matrix[merged_data['cluster'] == cluster_num]
        cluster_center = kmeans.cluster_centers_[cluster_num].reshape(1, -1)
        
        # 중심과 가장 유사한 설명 선택
        similarities = cosine_similarity(cluster_descriptions, cluster_center)
        best_index = similarities.argmax()

        # 'merged_data'에서 조건에 맞는 인덱스를 사용하여 설명 추출
        best_description = merged_data.loc[merged_data['cluster'] == cluster_num].iloc[best_index]['description']
        cluster_names.append(best_description[:50])  # 클러스터 이름을 설명 일부로 사용

    # 최종 데이터 프레임에 클러스터 이름 매핑
    merged_data['cluster_name'] = merged_data['cluster'].map(lambda x: cluster_names[x])

    # 필요한 결과 형식으로 반환
    result_data = merged_data[['packageName', 'cluster_name']]
    return result_data

# 실행 예제
if __name__ == "__main__":
    # JSON 파일과 CSV 파일 경로
    json_file = 'app_data_ee21702d85b4e078.json'
    csv_file = 'cleaned_apps_data.csv'
    
    # 앱 데이터 및 클린 데이터 로드
    app_data = load_app_data(json_file)
    cleaned_data = load_cleaned_apps_data(csv_file)
    
    # 앱 데이터와 클린 데이터 병합 및 클러스터링 수행
    merged_data = merge_app_descriptions(app_data, cleaned_data)
    
    # 불용어 추가 (예시로 'app', 'lite', 'file' 등을 추가)
    additional_stopwords = ['pdf', 'file', 'app', 'lite', 'application']
    
    # 클러스터링 실행
    result = perform_clustering(merged_data, n_clusters=4, additional_stopwords=additional_stopwords)
    
    # 결과 출력
    for item in result.itertuples():
        print(f"Package Name: {item.packageName}, Cluster Name: {item.cluster_name}")


c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Package Name: com.selabs.speak, Cluster Name: With the official ChatGPT app, get instant answers
Package Name: com.selabs.speak, Cluster Name: With the official ChatGPT app, get instant answers
Package Name: com.selabs.speak, Cluster Name: With the official ChatGPT app, get instant answers
Package Name: com.selabs.speak, Cluster Name: With the official ChatGPT app, get instant answers
Package Name: com.selabs.speak, Cluster Name: With the official ChatGPT app, get instant answers
Package Name: com.selabs.speak, Cluster Name: With the official ChatGPT app, get instant answers
Package Name: com.selabs.speak, Cluster Name: With the official ChatGPT app, get instant answers
Package Name: com.selabs.speak, Cluster Name: With the official ChatGPT app, get instant answers
Package Name: com.selabs.speak, Cluster Name: With the official ChatGPT app, get instant answers
Package Name: com.selabs.speak, Cluster Name: With the official ChatGPT app, get instant answers
Package Name: com.selabs.speak

C:\Users\user\AppData\Local\Temp\ipykernel_68124\2737557473.py:20: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(csv_file)
c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['android', 'apps', 'barcelona', 'chatgpt', 'chrome', 'com', 'easymover', 'gm', 'google', 'instagram', 'kakao', 'microsoft', 'nhn', 'office', 'officehubrow', 'openai', 'pinterest', 'sbrowser', 'search', 'sec', 'selabs', 'shealth', 'skydrive', 'speak', 'tachyon', 'talk', 'webtoon'] not in stop_words.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  su